# Assistant Rédacteur de Devis
<img src="redacteur_devis_base.png" alt="Logigramme" width="1000"/>

## Description du Projet


  
  Ce notebook décrit le processus de création d'un assistant virtuel capable de générer des devis basés sur les descriptions fournies par les utilisateurs. L'assistant utilise une base de données de clients et de produits pour produire un devis modifiable.


## Étapes du Projet

1. **Import des bibliothèques nécessaires**
2. **Configuration de la clé API OpenAI**
3. **Définition des modèles de données avec Pydantic**
4. **Fonction de génération de contexte pour les requêtes**
5. **Création de la base de données des clients et des produits**
6. **Initialisation du patch Instructor**
7. **Génération de réponse et traitement des données**
8. **Mise à jour du devis dans le fichier Excel**


### 1. Import des bibliothèques nécessaires

Nous commençons par importer les bibliothèques essentielles pour notre projet, notamment celles pour le traitement des données, l'intégration avec l'API OpenAI, et la manipulation des fichiers Excel.

In [1]:
import json
import csv
import datetime
import getpass
import os
import openpyxl
from openpyxl.utils import get_column_letter
from pydantic import BaseModel, Field
from openai import OpenAI
import instructor
import chromadb
import chromadb.utils.embedding_functions as embedding_functions

**OpenAI :** Utilisé pour l'intégration avec les modèles de langage avancés afin d'extraire les informations des descriptions fournies par les utilisateurs. Pour obtenir une clé API OpenAI, vous devez créer un compte sur la [plateforme OpenAI](https://beta.openai.com/signup/) et générer une clé API depuis votre tableau de bord. Cette clé est ensuite utilisée pour authentifier les requêtes à l'API.

**Openpyxl :** Bibliothèque Python pour lire et écrire des fichiers Excel, utilisée pour générer et modifier les devis.

**Chroma :** Utilisée pour la vectorisation des bases de données et l'embedding. Chroma facilite la recherche par similarité entre les documents en utilisant des embeddings de texte, qui sont des représentations vectorielles des textes. Pour ce faire, les embeddings sont générés à l'aide de la clé API OpenAI pour transformer les textes en vecteurs, ce qui permet de rechercher des correspondances similaires de manière efficace.

**Instructor :** Un patch pour le modèle de langage permettant de contrôler le format et le contenu de la sortie, assurant que les réponses générées sont bien structurées et répondent aux besoins spécifiques du contexte de création de devis.

### 2. Configuration de la clé API OpenAI

Pour utiliser les services OpenAI, nous devons configurer une clé API. Cette clé est stockée en tant que variable d'environnement pour des raisons de sécurité.

In [2]:
# Clé API OpenAI en variable d'environnement
def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")

_set_if_undefined("OPENAI_API_KEY")

### 3. Création de la base de données des clients et des produits

Nous créons une base de données fictive pour les clients et les produits. Ces données seront utilisées pour générer les devis. Les données sont ensuite vectorisées à l'aide de Chroma pour faciliter la recherche par similarité.

#### Clients

In [3]:
import csv

# Exemple de données fictives pour les clients
clients = [
    {"id": 1, "name": "Jean Dupont", "company": "ABC Corp", "email": "jean.dupont@abccorp.com", "phone": "0123456789"},
    {"id": 2, "name": "Marie Curie", "company": "XYZ Inc", "email": "marie.curie@xyzinc.com", "phone": "9876543210"},
    {"id": 3, "name": "Paul Martin", "company": "DEF Ltd", "email": "paul.martin@def.com", "phone": "5647382910"},
    {"id": 4, "name": "Lucie Bernard", "company": "GHI Solutions", "email": "lucie.bernard@ghi.com", "phone": "4738291020"},
    {"id": 5, "name": "Eric Petit", "company": "JKL Technologies", "email": "eric.petit@jkl.com", "phone": "3829104750"},
    {"id": 6, "name": "Sophie Durand", "company": "MNO Services", "email": "sophie.durand@mno.com", "phone": "2190384750"},
    {"id": 7, "name": "Alain Leroy", "company": "PQR Innovations", "email": "alain.leroy@pqr.com", "phone": "9038471200"},
    {"id": 8, "name": "Julie Lambert", "company": "STU Enterprises", "email": "julie.lambert@stu.com", "phone": "1209384750"},
    {"id": 9, "name": "Henri Moreau", "company": "VWX Solutions", "email": "henri.moreau@vwx.com", "phone": "3847201090"},
    {"id": 10, "name": "Claire Thomas", "company": "YZ Corp", "email": "claire.thomas@yz.com", "phone": "9182736450"},
    {"id": 11, "name": "Antoine Dubois", "company": "Alpha Inc", "email": "antoine.dubois@alpha.com", "phone": "5647382910"},
    {"id": 12, "name": "Elodie Dupuis", "company": "Beta Ltd", "email": "elodie.dupuis@beta.com", "phone": "1827364590"},
    {"id": 13, "name": "Michel Faure", "company": "Gamma Tech", "email": "michel.faure@gamma.com", "phone": "4738291020"},
    {"id": 14, "name": "Nathalie Girard", "company": "Delta Systems", "email": "nathalie.girard@delta.com", "phone": "5647382910"},
    {"id": 15, "name": "Vincent Morel", "company": "Epsilon Services", "email": "vincent.morel@epsilon.com", "phone": "9038471200"},
    {"id": 16, "name": "Isabelle Garnier", "company": "Zeta Innovations", "email": "isabelle.garnier@zeta.com", "phone": "1209384750"},
    {"id": 17, "name": "Marc Lefevre", "company": "Eta Enterprises", "email": "marc.lefevre@eta.com", "phone": "2190384750"},
    {"id": 18, "name": "Sandrine Mercier", "company": "Theta Solutions", "email": "sandrine.mercier@theta.com", "phone": "3847201090"},
    {"id": 19, "name": "Thierry Blanchard", "company": "Iota Technologies", "email": "thierry.blanchard@iota.com", "phone": "4738291020"},
    {"id": 20, "name": "Emilie Roche", "company": "Kappa Ltd", "email": "emilie.roche@kappa.com", "phone": "9182736450"}
]


#### Produits

In [4]:

# Exemple de données fictives pour les produits/services
products = [
    {"id": 1, "name": "Ordinateur Portable", "description": "Ordinateur portable 15 pouces, 8 Go RAM, 256 Go SSD", "price": 1000},
    {"id": 2, "name": "Imprimante", "description": "Imprimante multifonction, couleur, Wi-Fi", "price": 150},
    {"id": 3, "name": "Licence de logiciel de gestion de projet", "description": "Licence annuelle pour logiciel de gestion de projet", "price": 300},
    {"id": 4, "name": "Écran 27 pouces", "description": "Résolution 4K, anti-reflets", "price": 400},
    {"id": 5, "name": "Clavier mécanique", "description": "Switches Cherry MX Red, rétroéclairé", "price": 150},
    {"id": 6, "name": "Souris ergonomique", "description": "Sans fil, DPI ajustable", "price": 80},
    {"id": 7, "name": "Station d'accueil", "description": "5 ports USB, HDMI, Ethernet", "price": 250},
    {"id": 8, "name": "Casque audio", "description": "Réduction de bruit active, sans fil", "price": 200},
    {"id": 9, "name": "Routeur Wi-Fi", "description": "Dual-band, vitesse 1200 Mbps", "price": 120},
    {"id": 10, "name": "Disque dur externe", "description": "2 To, USB 3.0", "price": 100},
    {"id": 11, "name": "Microphone USB", "description": "Capture audio haute qualité", "price": 70},
    {"id": 12, "name": "Webcam HD", "description": "1080p, micro intégré", "price": 90},
    {"id": 13, "name": "Tablette graphique", "description": "Sensibilité à la pression 8192 niveaux", "price": 350},
    {"id": 14, "name": "Adaptateur USB-C", "description": "HDMI, USB 3.0, Ethernet", "price": 60},
    {"id": 15, "name": "Onduleur", "description": "1500 VA, écran LCD", "price": 180},
    {"id": 16, "name": "Projecteur", "description": "3000 lumens, résolution 1080p", "price": 500},
    {"id": 17, "name": "Clé USB", "description": "128 Go, USB 3.1", "price": 30},
    {"id": 18, "name": "Logiciel antivirus", "description": "Protection complète, mise à jour automatique", "price": 50},
    {"id": 19, "name": "Serveur NAS", "description": "4 baies, RAID supporté", "price": 600},
    {"id": 20, "name": "Logiciel de comptabilité", "description": "Gestion des finances, rapports détaillés", "price": 400},
    {"id": 21, "name": "Câble HDMI", "description": "2 mètres, support 4K", "price": 20},
    {"id": 22, "name": "Support pour moniteur", "description": "Réglable en hauteur", "price": 60},
    {"id": 23, "name": "Lecteur de cartes mémoire", "description": "Supporte SD, microSD, CF", "price": 25},
    {"id": 24, "name": "Chargeur sans fil", "description": "Compatibilité Qi", "price": 40},
    {"id": 25, "name": "Switch réseau", "description": "8 ports Gigabit", "price": 70},
    {"id": 26, "name": "Hub USB", "description": "4 ports USB 3.0", "price": 35},
    {"id": 27, "name": "Tapis de souris", "description": "Surface en tissu, base antidérapante", "price": 15},
    {"id": 28, "name": "Batterie externe", "description": "10000 mAh, double port USB", "price": 50},
    {"id": 29, "name": "Système de conférence", "description": "Microphone et haut-parleur intégrés", "price": 300},
    {"id": 30, "name": "Logiciel de création graphique", "description": "Outils avancés, support vectoriel", "price": 250},
    {"id": 31, "name": "Module de mémoire RAM", "description": "8 Go DDR4, 3200 MHz", "price": 100}
]


#### Utilisation des Embeddings et Structuration des Collections

Dans cette étape, nous utilisons des embeddings pour transformer les textes en représentations vectorielles, facilitant ainsi la recherche de similarités entre les documents. Les embeddings sont des représentations numériques des textes qui capturent les significations et les nuances du langage de manière compacte et mathématiquement manipulable.

**ChromaDB** est utilisée pour gérer les collections de documents et leurs métadonnées, permettant d'organiser les données clients et produits de manière structurée. Nous utilisons la bibliothèque `chromadb` pour créer une base de données persistante, où chaque document (nom de client ou produit) est associé à un vecteur d'embedding et à des métadonnées pertinentes.

1. **Embeddings OpenAI** : Les embeddings sont générés en utilisant la clé API OpenAI avec le modèle `text-embedding-ada-002`. Cette représentation vectorielle est ensuite stockée dans les collections pour chaque document.

2. **Création des Collections** : 
   - **Clients** : Les informations de chaque client (nom, société, email, téléphone) sont stockées sous forme de métadonnées, avec le nom du client comme document principal.
   - **Produits** : Les détails des produits (nom, description, prix) sont stockés sous forme de métadonnées, avec le nom du produit comme document principal.

3. **Recherche de Similarités** : En structurant les données de cette manière, nous pouvons utiliser les embeddings pour effectuer des recherches par similarité vectorielle, permettant de retrouver facilement les clients ou produits correspondant à une requête spécifique. Cela est particulièrement utile pour suggérer des produits similaires ou compléter les informations manquantes dans les devis.



In [5]:
def create_database(clients, products):
    # Initialisation du client ChromaDB avec un chemin pour stocker la base de données
    client = chromadb.PersistentClient(path="new_chroma_db")
    
    # Configuration de l'utilisation des embeddings OpenAI
    openai_ef = embedding_functions.OpenAIEmbeddingFunction(api_key=os.environ["OPENAI_API_KEY"], model_name="text-embedding-ada-002")

    # Création de la collection pour les produits avec embeddings
    collection_product = client.get_or_create_collection(name="products", embedding_function=openai_ef)
    
    # Création de la collection pour les clients avec embeddings
    collection_client = client.get_or_create_collection(name="clients", embedding_function=openai_ef)

    # Ajout des informations sur les produits à la collection de produits
    collection_product.add(
        ids=[str(p['id']) for p in products], 
        documents=[p['name'] for p in products], 
        metadatas=[{'name': p['name'], 'description': p['description'], 'price': p['price']} for p in products]
    )

    # Ajout des informations sur les clients à la collection de clients
    collection_client.add(
        ids=[str(c['id']) for c in clients], 
        documents=[c['name'] for c in clients], 
        metadatas=[{'name': c['name'], 'company': c['company'], 'email': c['email'], 'phone': c['phone']} for c in clients]
    )

    # Retourne les collections créées
    return collection_product, collection_client

# Création des collections de produits et clients à partir des données fournies
collection_product, collection_client = create_database(clients, products)


Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Add of existing embedding ID: 4
Add of existing embedding ID: 5
Add of existing embedding ID: 6
Add of existing embedding ID: 7
Add of existing embedding ID: 8
Add of existing embedding ID: 9
Add of existing embedding ID: 10
Add of existing embedding ID: 11
Add of existing embedding ID: 12
Add of existing embedding ID: 13
Add of existing embedding ID: 14
Add of existing embedding ID: 15
Add of existing embedding ID: 16
Add of existing embedding ID: 17
Add of existing embedding ID: 18
Add of existing embedding ID: 19
Add of existing embedding ID: 20
Add of existing embedding ID: 21
Add of existing embedding ID: 22
Add of existing embedding ID: 23
Add of existing embedding ID: 24
Add of existing embedding ID: 25
Add of existing embedding ID: 26
Add of existing embedding ID: 27
Add of existing embedding ID: 28
Add of existing embedding ID: 29
Add of existing embedding ID: 30
Add of existing emb

### 4. Fonction de génération de contexte pour les requêtes

La fonction create_context est conçue pour générer un contexte riche et informatif en utilisant les données des clients et des produits. Ce contexte est ensuite utilisé pour guider les réponses du modèle de langage. Elle réalise des requêtes sur les bases de données clients et produits en fonction des entrées fournies, et retourne une vue d'ensemble formatée des informations pertinentes.

In [6]:
def create_context(inputs, client_retriever, product_retriever):
    """
    Génère un contexte formaté à partir des données client et produit.

    Args:
        inputs (str): Le texte d'entrée pour la requête.
        client_retriever: L'objet permettant de récupérer les données clients.
        product_retriever: L'objet permettant de récupérer les données produits.

    Returns:
        str: Le contexte formaté incluant les informations du client et des produits.
    """
    # Requêtes pour récupérer les données clients et produits
    client_data = client_retriever.query(query_texts=[inputs], n_results=1)
    product_data = product_retriever.query(query_texts=[inputs], n_results=3)

    # Formate les données du client en un JSON bien présenté
    client_context = f"Client: {json.dumps(client_data['metadatas'][0], indent=4)}"

    # Formate les données des produits en un JSON bien présenté pour chaque produit
    product_context = "\n".join([json.dumps(product, indent=4) for product in product_data['metadatas']])

    # Retourne le contexte complet combinant les informations du client et des produits
    return f"{client_context}\n\nProduits:\n{product_context}"


In [7]:
# Exemple d'utilisation de la fonction create_context
## Modifiez la variable inputs pour tester différentes requêtes et observer comment le système réagit
inputs = "Jean Dupont souhaite acheter un ordinateur portable et une imprimante."

context = create_context(inputs, collection_client, collection_product)
print(context)


Client: [
    {
        "company": "ABC Corp",
        "email": "jean.dupont@abccorp.com",
        "name": "Jean Dupont",
        "phone": "0123456789"
    }
]

Produits:
[
    {
        "description": "Imprimante multifonction, couleur, Wi-Fi",
        "name": "Imprimante",
        "price": 150
    },
    {
        "description": "Ordinateur portable 15 pouces, 8 Go RAM, 256 Go SSD",
        "name": "Ordinateur Portable",
        "price": 1000
    },
    {
        "description": "3000 lumens, r\u00e9solution 1080p",
        "name": "Projecteur",
        "price": 500
    }
]


### 5. Définition des modèles de données avec Pydantic

Pydantic est une bibliothèque pour la validation des données et la gestion des modèles de données. Nous définissons des modèles pour les clients, les produits, les suggestions de produits, et les réponses complètes du système.

In [8]:
# Définition des modèles de données
class Client(BaseModel):
    name: str
    company: str
    email: str
    phone: str

class Product(BaseModel):
    name: str
    price: float
    description: str
    quantity: int
    
class Suggestions_product(BaseModel):
    name: str
    price: float
    description: str

class Property(BaseModel):
    key: str = Field(description="Nom client ou nom produit")
    value: str = Field(description="Information présente ou absente de la base de donnée")

class Response(BaseModel):
    Message_client: str = Field(description="Informe sur les informations non trouvés dans la base de données")
    Message_produit: str = Field(description="Informe sur les informations non trouvés dans la base de données")
    client: Client
    products: list[Product]
    properties: list[Property]
    suggestion : list[Suggestions_product] = Field(description="Suggestions de produits du catalogue")

### 6. Initialisation du patch Instructor

Dans cette étape, nous utilisons Instructor pour intégrer le client OpenAI. L'objectif est de garantir que les données générées par le modèle correspondent aux objets Pydantic définis précédemment. Si les données de sortie ne correspondent pas, Instructor renvoie un message d'erreur au modèle, qui peut alors ajuster la réponse. Cela permet d'assurer la cohérence et l'exactitude des informations, améliorant ainsi la qualité des devis générés.


In [9]:
# Patch the OpenAI client
client = instructor.from_openai(OpenAI())


Avec cette configuration, toutes les requêtes passent par Instructor, qui vérifie la conformité des données avec les modèles Pydantic. Ce mécanisme est essentiel pour maintenir l'intégrité des données et fournir des résultats précis et fiables.

### 7. Génération de réponse et traitement des données

Nous utilisons l'API OpenAI pour traiter le message du client et générer une réponse structurée sous forme de devis.

In [10]:
def generate_response(inputs, client_retriever, product_retriever):
    # Créer le contexte en récupérant les données clients et produits
    context = create_context(inputs, client_retriever, product_retriever)
       
    # Utiliser Instructor pour traiter le message et valider les informations
    response = client.chat.completions.create(
        model="gpt-4o",
        response_model=Response,
        max_retries=3,
        messages=[
            {
                "role": "system",
                "content": """
                Vous êtes un assistant virtuel spécialisé dans la création de devis pour les clients.
                Votre tâche est d'analyser les messages entrants des clients pour en extraire les informations pertinentes et remplir les champs nécessaires pour créer un devis.
                Confrontez les noms et les produits annoncés au contexte fourni, qui comprend une base de données clients et un catalogue produits.
                """,
            },
            {"role": "user", "content": context},
            {"role": "user", "content": f"Question: Create a quote for customer {inputs} for product {inputs}."},
        ],
        validation_context={"text_chunk": context},
    )
    return response


#### Exemple

In [15]:
inputs = """
Bonjour, je m'appelle Jean Dupont et je travaille pour ABC Corp.
Je souhaiterais commander un ordinateur portable et trois imprimantes.
"""

response = generate_response(inputs, collection_client, collection_product)
print(json.dumps(response.dict(), indent=4))

{
    "Message_client": "ABC Corp et Jean Dupont trouv\u00e9s dans la base de donn\u00e9es.",
    "Message_produit": "Produit Support pour moniteur non trouv\u00e9 dans la demande client.",
    "client": {
        "name": "Jean Dupont",
        "company": "ABC Corp",
        "email": "jean.dupont@abccorp.com",
        "phone": "0123456789"
    },
    "products": [
        {
            "name": "Imprimante",
            "price": 150.0,
            "description": "Imprimante multifonction, couleur, Wi-Fi",
            "quantity": 3
        },
        {
            "name": "Ordinateur Portable",
            "price": 1000.0,
            "description": "Ordinateur portable 15 pouces, 8 Go RAM, 256 Go SSD",
            "quantity": 1
        }
    ],
    "properties": [
        {
            "key": "Client",
            "value": "ABC Corp"
        },
        {
            "key": "Client",
            "value": "Jean Dupont"
        },
        {
            "key": "Produit",
            "value"

### 8. Mise à jour du devis dans le fichier Excel


La réponse est transformée en un format de données utilisable pour remplir le devis.

In [12]:
def transform_response_to_data(response: Response):
    data = {
        'nom_personne': response.client.name,
        'nom_entreprise': response.client.company,
        'tel': response.client.phone,
        'email_client': response.client.email,
        'num_fact': '2024-001',  # Numéro de facture généré ou fourni
        'date': datetime.datetime.now().strftime("%Y-%m-%d"),
        'items': [{'description': product.name, 'qtt': product.quantity, 'prix_unitaire': product.price} for product in response.products]
    }
    return data

 En suite, cette fonction met à jour un fichier Excel existant avec les informations du devis généré. Les détails du client, des produits et les totaux sont ajoutés ou mis à jour dans le fichier.

In [13]:
def update_invoice(file_path, output_path, data):
    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook.active

    sheet['B11'] = data['nom_personne']
    sheet['B12'] = data['nom_entreprise']
    sheet['B13'] = data['tel']
    sheet['B14'] = data['email_client']
    sheet['F11'] = data['num_fact']
    sheet['F14'] = data['date']

    for idx, item in enumerate(data['items']):
        row = 19 + idx
        sheet[f'B{row}'] = item['description']
        sheet[f'E{row}'] = item['qtt']
        sheet[f'F{row}'] = item['prix_unitaire']
        sheet[f'G{row}'] = item['qtt'] * item['prix_unitaire']

    workbook.save(output_path)
    print('Le document de facturation a été créé avec succès.')

In [14]:
data = transform_response_to_data(response)
update_invoice('Devis_AVmaker.xlsx', 'facture_AVmaker_modifiee.xlsx', data)

Le document de facturation a été créé avec succès.
